In [2]:
from gurobipy import *
import pandas as pd
import numpy as np

## Parameters

In [12]:
df_county = pd.read_csv('data/county_code.csv')
df_market = pd.read_csv('data/market_code.csv')
I = range(df_county.shape[0])
J = range(df_market.shape[0])

In [13]:
df_demand = pd.read_excel("data/demand_function.xlsx")
A = df_demand["slope"]
B = df_demand["intercept"]
# df_demand

In [14]:
# 旺季產量
df_production = pd.read_csv("data/county_production.csv")
# 旺季一個月
df_production1 = df_production.copy()
df_production1['quantity'] = df_production1['quantity'] * 0.6 / 4 
# 每天
df_production1['quantity'] /= 30
Q_s = df_production1['quantity']
df_production1

,Unnamed: 0,name,quantity,code
0,0,新北市,15366.243889,0
1,1,台北市,2748.499444,1
2,2,桃園市,16033.365000,2
3,3,台中市,176370.740556,3
4,4,台南市,95918.435556,4
5,5,高雄市,38387.124444,5
6,6,宜蘭縣,431689.425000,6
7,7,新竹縣,37136.298333,7
8,8,苗栗縣,23024.741667,8
9,9,彰化縣,226593.761111,9


In [131]:
L = 2500
C_t = 0.03
C_p = 7.5
M = 4300

In [108]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

In [17]:
Q_s.sum()

2012703.3902777776

In [18]:
df_s2m = pd.read_excel("data/s2m.xlsx")
D = df_s2m.iloc[:, 1:]
D

,新北市,台北市,桃園市,台中市,台南市,高雄市,宜蘭縣,新竹縣,苗栗縣,彰化縣,南投縣,雲林縣,嘉義縣,屏東縣,台東縣,花蓮縣,基隆市,新竹市,嘉義市,桃園縣
0,0.000,0.003,42.823,170.535,313.911,358.839,62.858,79.306,117.277,186.964,238.516,228.744,271.751,382.391,317.048,155.954,27.893,85.772,257.907,42.728
1,0.000,0.003,42.823,170.535,313.911,358.839,62.858,79.306,117.277,186.964,238.516,228.744,271.751,382.391,317.048,155.954,27.893,85.772,257.907,42.728
2,11.127,11.130,31.816,158.900,302.276,347.203,70.750,68.267,105.642,175.328,216.388,217.109,260.116,370.756,324.940,163.847,30.098,74.733,246.271,31.721
3,44.193,44.196,4.016,130.903,274.278,319.206,103.057,40.270,77.644,147.331,188.391,189.111,232.119,342.758,476.367,196.154,58.571,46.736,218.274,3.921
4,164.690,164.692,137.656,0.000,154.034,198.962,222.960,96.079,57.807,27.087,54.103,68.867,111.874,220.274,353.882,316.056,185.738,94.463,97.015,137.561
5,199.473,199.475,172.439,46.528,117.162,162.089,257.743,130.862,92.590,22.542,61.397,31.995,75.002,185.642,319.250,350.839,220.521,129.246,61.158,172.344
6,193.115,193.117,166.081,40.170,118.541,163.468,251.385,124.504,86.232,16.184,77.351,33.374,76.381,187.021,320.629,344.481,214.163,122.888,62.537,165.986
7,207.661,207.663,180.706,30.635,138.342,183.987,265.931,139.128,90.819,40.615,53.691,63.020,97.487,192.504,326.112,210.713,228.788,133.220,69.245,180.610
8,215.161,215.164,188.127,62.216,100.610,145.538,273.431,146.550,108.279,38.230,99.397,9.296,58.450,169.090,302.699,366.528,236.210,144.934,44.606,188.032
9,352.581,352.583,325.547,199.635,49.564,0.000,410.851,283.970,245.698,175.650,235.563,136.072,110.620,32.328,165.768,331.925,373.629,282.353,114.056,325.451


In [47]:
meanDis = []
for i, row in D.iterrows():
    meanDis.append(row.mean())
# meanDis

In [61]:
Q_d

0     157183.655977
1     378746.411079
2      50967.553936
3      32963.338192
4      42682.312865
5      56868.415441
6      84505.971098
7       2919.810496
8     379836.636637
9      79209.460641
10     24236.137026
11     33785.737609
12     25497.405917
13     14901.787582
14     20436.245353
15     40815.417040
16     88827.083436
Name: quantity, dtype: float64

- Formulation1
$$
\begin{split}
 \mbox{max} \quad & \sum_{j \in J}p_j\sum_{i \in I}x_{ij} - C\sum_{i \in I}\sum_{j \in J}D_{ij}(My_{ij} + x_{ij}) \quad \mbox{(revenue)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & p_j = A_j \sum_{i \in I} x_{ij} + B_j \quad \forall j \in J \\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [132]:
model = Model('solution1')
model.setParam('NonConvex', 2)

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.INTEGER, name = 'y')
p = model.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = 'p')

# objective function
model.setObjective(quicksum(p[j] * quicksum(x[i, j] for i in I) for j in J) 
                            - C_t * quicksum(D.iloc[j, i] * (M * y[i, j] + x[i, j]) for i in I for j in J), GRB.MAXIMIZE)

# constraints (for both single and multiple sourcing)
constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)
constr = model.addConstrs(p[j] == A[j] * quicksum(x[i, j] for i in I) + B[j] for j in J)

model.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 697 columns and 1377 nonzeros
Model fingerprint: 0x80738840
Model has 340 quadratic objective terms
Variable types: 357 continuous, 340 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-05, 2e+03]
  Objective range  [9e-05, 6e+04]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+05]
Found heuristic solution: objective -1.62110e+16
Presolve removed 4 rows and 4 columns
Presolve time: 0.04s
Presolved: 1054 rows, 1034 columns, 3070 nonzeros
Presolved model has 340 bilinear constraint(s)
Found heuristic solution: objective -0.0000000
Variable types: 698 continuous, 336 integer (51 binary)

Root relaxation: objective 4.764278e+07, 856 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Cur

In [137]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)
df_result_production["production"] = p
df_result_production

,Unnamed: 0,code,name,production
0,0,0,新北市,15366.243889
1,1,1,台北市,2748.499444
2,2,2,桃園市,16033.365000
3,3,3,台中市,176370.740556
4,4,4,台南市,95000.000000
5,5,5,高雄市,38387.124444
6,6,6,宜蘭縣,431689.425000
7,7,7,新竹縣,37136.298333
8,8,8,苗栗縣,22500.000000
9,9,9,彰化縣,182500.000000


In [139]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

In [140]:
df_result_consumption = df_market.copy()
c = []
for j in J:
    total = 0
    for i in I:
        total += x[i, j].x
    c.append(total)
df_result_consumption["consumption"] = c
df_result_consumption

,Unnamed: 0,code,name,consumption
0,0,0,台北二綜合農產品批發市場,138593.803333
1,1,1,台北一綜合農產品批發市場,284520.940000
2,2,2,三重區綜合農產品批發市場,32500.000000
3,3,3,桃農綜合農產品批發市場,71272.925556
4,4,4,台中市綜合農產品批發市場,63870.740556
5,5,5,永靖鄉綜合農產品批發市場,50000.000000
6,6,6,溪湖鎮綜合農產品批發市場,107500.000000
7,7,7,南投市綜合農產品批發市場,2500.000000
8,8,8,西螺鎮綜合農產品批發市場,458767.212222
9,9,9,高雄市綜合農產品批發市場,128387.124444


In [141]:
Q_d

0     157183.655977
1     378746.411079
2      50967.553936
3      32963.338192
4      42682.312865
5      56868.415441
6      84505.971098
7       2919.810496
8     379836.636637
9      79209.460641
10     24236.137026
11     33785.737609
12     25497.405917
13     14901.787582
14     20436.245353
15     40815.417040
16     88827.083436
Name: quantity, dtype: float64

In [133]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)
    elif var.varName.startswith('y'):
        print(var.varName, '=', var.x)
    elif var.varName.startswith('p'):
        print(var.varName, '=', var.x)

x[0,0] = 1093.8033333520748
x[0,1] = 14272.44055553672
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 0.0
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 0.0
x[0,11] = 0.0
x[0,12] = 0.0
x[0,13] = 0.0
x[0,14] = 0.0
x[0,15] = 0.0
x[0,16] = 0.0
x[1,0] = 0.0
x[1,1] = 2748.499444444347
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 0.0
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 0.0
x[1,11] = 0.0
x[1,12] = 0.0
x[1,13] = 0.0
x[1,14] = 0.0
x[1,15] = 0.0
x[1,16] = 0.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 9999.999999998861
x[2,3] = 6033.365000001023
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 0.0
x[2,7] = 0.0
x[2,8] = 0.0
x[2,9] = 0.0
x[2,10] = 0.0
x[2,11] = 0.0
x[2,12] = 0.0
x[2,13] = 0.0
x[2,14] = 0.0
x[2,15] = 0.0
x[2,16] = 0.0
x[3,0] = 0.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 63870.740555554265
x[3,5] = 0.0
x[3,6] = 0.0
x[3,7] = 0.0
x[3,8] = 0.0
x[3,9] = 0.0
x[3,10] = 0.0
x[3,11] = 0.0
x[3,12] = 0.0
x[3,13] = 0.0
x[3,14] = 0.0
x[3,15] = 42500.00000000001
x[

In [134]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)

In [135]:
p

[15366.243888888794,
 2748.499444444347,
 16033.364999999885,
 176370.74055555556,
 94999.9999999998,
 38387.12444444444,
 431689.4249999998,
 37136.29833333321,
 22499.999999999865,
 182500.00000000032,
 0.0,
 458767.21222222276,
 0.0,
 74353.7016666668,
 31505.104999999996,
 19572.55277777778,
 0.0,
 603.2622222220944,
 0.0,
 12499.9999999999]

In [136]:
df_demand

,Unnamed: 0,ID,slope,intercept
0,台北二,104,-0.00006,30.275
1,台北一,109,-0.00003,30.715
2,三重區,241,-0.00020,27.579
3,桃農,338,-0.00010,26.032
4,台中市,400,-0.00020,26.975
5,永靖鄉,512,-0.00020,22.064
6,溪湖鎮,514,-0.00010,22.777
7,南投市場,540,-0.00440,28.924
8,西螺鎮,648,-0.00003,29.549
9,高雄市,800,-0.00010,29.729


In [30]:
# consumption
df_consumption = pd.read_csv("data/consumption.csv")
df_consumption = df_consumption.replace("南投市", "南投市場")
df_consumption = df_consumption[df_consumption["market_name"].isin(list(df_demand["Unnamed: 0"]))]
df_consumption["month"] = df_consumption["date"].str.split('.').str[1]
consumptions = []
month = df_consumption["month"].unique()
for i in month:
    df = df_consumption[df_consumption["month"] == i][["market_name", 
                        "quantity", "mean_price"]].groupby(["market_name"]).mean()
    df = df_demand[["Unnamed: 0"]].merge(right = df, right_on = "market_name", left_on = "Unnamed: 0")
    consumptions.append(df)

print(consumptions[0])

   Unnamed: 0       quantity  mean_price
0         台北二  157183.655977   13.280257
1         台北一  378746.411079   13.322492
2         三重區   50967.553936   12.080210
3          桃農   32963.338192   12.052526
4         台中市   42682.312865   11.989855
5         永靖鄉   56868.415441    9.967664
6         溪湖鎮   84505.971098   12.446673
7        南投市場    2919.810496   14.902399
8         西螺鎮  379836.636637   11.044560
9         高雄市   79209.460641   15.881483
10        鳳山區   24236.137026   11.331352
11        屏東市   33785.737609   12.140778
12        花蓮市   25497.405917   15.766917
13        台東市   14901.787582   15.333606
14        宜蘭市   20436.245353    9.801510
15        豐原區   40815.417040   10.716375
16        板橋區   88827.083436   12.826775


In [31]:
P = consumptions[0].mean_price
Q_d = consumptions[0].quantity

In [32]:
Q_d.sum()

1514383.380325805

- Formulation2
$$
\begin{split}
 \mbox{min} \quad & C\sum_{i \in I}\sum_{j \in J}D_{ij}(My_{ij} + x_{ij}) \quad \mbox{(revenue)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & \sum_i x_{ij} \geq Q^d_j \quad \forall j \in J \quad \mbox{(lower limit of demand quantity)}\\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [98]:
# Calcalate directly from real price
model = Model('solution1-1')

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'y')

# objective function
model.setObjective(C_t * quicksum(D.iloc[j, i] * (M * y[i, j] + x[i, j]) for i in I for j in J), GRB.MINIMIZE)

# constraints (for both single and multiple sourcing)
constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(quicksum(x[i, j] for i in I) >= Q_d[j] for j in J)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)
# constr = model.addConstrs(p[j] == A[j] * quicksum(x[i, j] for i in I) + B[j] for j in J)

model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 680 columns and 1360 nonzeros
Model fingerprint: 0x5e1e78da
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e-04, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+05]
Presolve removed 340 rows and 340 columns
Presolve time: 0.01s
Presolved: 37 rows, 340 columns, 680 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.772435e+05   0.000000e+00      0s
      44    1.3878008e+07   0.000000e+00   0.000000e+00      0s

Solved in 44 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.387800762e+07


In [65]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)
    if var.varName.startswith('y'):
        print(var.varName, '=', var.x)

x[0,0] = 15366.24388888889
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 0.0
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 0.0
x[0,11] = 0.0
x[0,12] = 0.0
x[0,13] = 0.0
x[0,14] = 0.0
x[0,15] = 0.0
x[0,16] = 0.0
x[1,0] = 2748.499444444444
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 0.0
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 0.0
x[1,11] = 0.0
x[1,12] = 0.0
x[1,13] = 0.0
x[1,14] = 0.0
x[1,15] = 0.0
x[1,16] = 0.0
x[2,0] = 16033.364999999998
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 0.0
x[2,7] = 0.0
x[2,8] = 0.0
x[2,9] = 0.0
x[2,10] = 0.0
x[2,11] = 0.0
x[2,12] = 0.0
x[2,13] = 0.0
x[2,14] = 0.0
x[2,15] = 0.0
x[2,16] = 0.0
x[3,0] = 4045.927214116884
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 42682.31286549708
x[3,5] = 0.0
x[3,6] = 0.0
x[3,7] = 0.0
x[3,8] = 0.0
x[3,9] = 0.0
x[3,10] = 0.0
x[3,11] = 0.0
x[3,12] = 0.0
x[3,13] = 0.0
x[3,14] = 0.0
x[3,15] = 40815.41704035874
x[3,16] = 88827.0

In [66]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

In [67]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)
df_result_production["production"] = p
df_result_production

,Unnamed: 0,code,name,production
0,0,0,新北市,15366.243889
1,1,1,台北市,2748.499444
2,2,2,桃園市,16033.365000
3,3,3,台中市,176370.740556
4,4,4,台南市,24490.509166
5,5,5,高雄市,38387.124444
6,6,6,宜蘭縣,431689.425000
7,7,7,新竹縣,37136.298333
8,8,8,苗栗縣,23024.741667
9,9,9,彰化縣,226593.761111


In [126]:
Q_d

0     157183.655977
1     378746.411079
2      50967.553936
3      32963.338192
4      42682.312865
5      56868.415441
6      84505.971098
7       2919.810496
8     379836.636637
9      79209.460641
10     24236.137026
11     33785.737609
12     25497.405917
13     14901.787582
14     20436.245353
15     40815.417040
16     88827.083436
Name: quantity, dtype: float64

In [127]:
df_result_consumption = df_market.copy()
c = []
for j in J:
    total = 0
    for i in I:
        total += x[i, j].x
    c.append(total)
df_result_consumption["consumption"] = c
df_result_consumption

,Unnamed: 0,code,name,consumption
0,0,0,台北二綜合農產品批發市場,157183.655977
1,1,1,台北一綜合農產品批發市場,378746.411079
2,2,2,三重區綜合農產品批發市場,50967.553936
3,3,3,桃農綜合農產品批發市場,32963.338192
4,4,4,台中市綜合農產品批發市場,42682.312865
5,5,5,永靖鄉綜合農產品批發市場,56868.415441
6,6,6,溪湖鎮綜合農產品批發市場,84505.971098
7,7,7,南投市綜合農產品批發市場,2919.810496
8,8,8,西螺鎮綜合農產品批發市場,379836.636637
9,9,9,高雄市綜合農產品批發市場,79209.460641


### Step2

In [ ]:
- Decision Variables


- Formulation3
$$
\begin{split}
 \mbox{min} \quad & C^t\sum_{i \in I}\sum_{j \in J}D_{ij}(My_{ij} + x_{ij}) + \sum_{j \in J} d_j + 
 \sum_{i \in I} C_i \sum_{j \in J} x_{ij}\quad \mbox{(carbon footprint)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & \sum_i x_{ij} \geq Q^d_j \quad \forall j \in J \quad \mbox{(lower limit of demand quantity)}\\
 & d_j \geq C_j - \sum_{i \in I}x_{ij} \quad \forall j \in J \\
 & d_j \geq \sum_{i \in I}x_{ij} - C_j \quad \forall j \in J \\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [120]:
df_cost = pd.read_excel("data/cost.xlsx")
df_cost

,Unnamed: 0,name,quantity,code,Unnamed: 4,mc,Unnamed: 6,Unnamed: 7,Unnamed: 8,cheap,3.8
0,0,0,新北市,15366.243889,0,4.75,NaN,NaN,NaN,expensive,4.75
1,1,1,台北市,2748.499444,1,4.75,NaN,NaN,NaN,NaN,NaN
2,2,2,桃園市,16033.365000,2,4.75,NaN,NaN,NaN,NaN,NaN
3,3,3,台中市,176370.740556,3,4.75,NaN,NaN,NaN,NaN,NaN
4,4,4,台南市,95918.435556,4,3.80,NaN,NaN,NaN,NaN,NaN
5,5,5,高雄市,38387.124444,5,4.75,NaN,NaN,NaN,NaN,NaN
6,6,6,宜蘭縣,431689.425000,6,3.80,NaN,NaN,NaN,NaN,NaN
7,7,7,新竹縣,37136.298333,7,4.75,NaN,NaN,NaN,NaN,NaN
8,8,8,苗栗縣,23024.741667,8,4.75,NaN,NaN,NaN,NaN,NaN
9,9,9,彰化縣,226593.761111,9,3.80,NaN,NaN,NaN,NaN,NaN


In [112]:
quantity = []
for j in J:
    p = C_t * meanDis[j] + C_p
    print(p)
    quantity.append((p - B[j]) / A[j])
quantity

12.51183
12.51183
12.327627
12.206243999999998
11.7957
12.0124455
11.936411999999997
12.0613155
12.168822
14.255766000000001
14.239851000000002
14.4780135
14.932682999999999
16.642125
13.3898595
11.77101
12.508436999999997


[296052.8333333333,
 606772.3333333334,
 76256.865,
 138257.56000000003,
 75896.5,
 50257.7725,
 108405.88000000003,
 3832.428295454545,
 579339.2666666667,
 154732.33999999997,
 20360.298,
 51524.93250000001,
 37847.19500000001,
 52784.37500000001,
 26658.772142857146,
 72099.94999999998,
 151961.8111111111]

In [127]:
Q = quantity
C = df_cost["mc"]

# Calcalate directly from real price
model = Model('solution1-3')

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'y')
d = model.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = 'd')

# objective function
model.setObjective(C_t * quicksum(D.iloc[j, i] * (M * y[i, j] + x[i, j]) for i in I for j in J) 
                    + quicksum(d[j] for j in J) + quicksum(C[i] * quicksum(x[i, j] for j in J) for i in I), GRB.MINIMIZE)


constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(quicksum(x[i, j] for i in I) >= Q_d[j] for j in J)
constr = model.addConstrs(d[j] >= C[j] - quicksum(x[i, j] for i in I) for j in J)
onstr = model.addConstrs(d[j] >= quicksum(x[i, j] for i in I) - C[j] for j in J)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)
# constr = model.addConstrs(p[j] == A[j] * quicksum(x[i, j] for i in I) + B[j] for j in J)

model.optimize()



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 411 rows, 697 columns and 2074 nonzeros
Model fingerprint: 0x3978556a
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e-01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+05]
Presolve removed 340 rows and 340 columns
Presolve time: 0.01s
Presolved: 71 rows, 357 columns, 1394 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.893073e+05   0.000000e+00      0s
      75    1.6005280e+07   0.000000e+00   0.000000e+00      0s

Solved in 75 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.600528007e+07


In [128]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)
    if var.varName.startswith('y'):
        print(var.varName, '=', var.x)
    if var.varName.startswith('d'):
        print(var.varName, '=', var.x)

x[0,0] = 15366.24388888889
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 0.0
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 0.0
x[0,11] = 0.0
x[0,12] = 0.0
x[0,13] = 0.0
x[0,14] = 0.0
x[0,15] = 0.0
x[0,16] = 0.0
x[1,0] = 2748.499444444444
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 0.0
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 0.0
x[1,11] = 0.0
x[1,12] = 0.0
x[1,13] = 0.0
x[1,14] = 0.0
x[1,15] = 0.0
x[1,16] = 0.0
x[2,0] = 0.0
x[2,1] = 16033.364999999998
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 0.0
x[2,7] = 0.0
x[2,8] = 0.0
x[2,9] = 0.0
x[2,10] = 0.0
x[2,11] = 0.0
x[2,12] = 0.0
x[2,13] = 0.0
x[2,14] = 0.0
x[2,15] = 0.0
x[2,16] = 0.0
x[3,0] = 4045.927214116884
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 42682.31286549708
x[3,5] = 0.0
x[3,6] = 0.0
x[3,7] = 0.0
x[3,8] = 0.0
x[3,9] = 0.0
x[3,10] = 0.0
x[3,11] = 0.0
x[3,12] = 0.0
x[3,13] = 0.0
x[3,14] = 0.0
x[3,15] = 40815.41704035874
x[3,16] = 88827.0

In [130]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)
df_result_production["production"] = p
df_result_production

,Unnamed: 0,code,name,production
0,0,0,新北市,15366.243889
1,1,1,台北市,2748.499444
2,2,2,桃園市,16033.365000
3,3,3,台中市,176370.740556
4,4,4,台南市,24490.509166
5,5,5,高雄市,38387.124444
6,6,6,宜蘭縣,431689.425000
7,7,7,新竹縣,37136.298333
8,8,8,苗栗縣,23024.741667
9,9,9,彰化縣,226593.761111


In [125]:
df_result_consumption = df_market.copy()
c = []
for j in J:
    total = 0
    for i in I:
        total += x[i, j].x
    c.append(total)
df_result_consumption["consumption"] = c
df_result_consumption

,Unnamed: 0,code,name,consumption
0,0,0,台北二綜合農產品批發市場,157183.655977
1,1,1,台北一綜合農產品批發市場,378746.411079
2,2,2,三重區綜合農產品批發市場,50967.553936
3,3,3,桃農綜合農產品批發市場,32963.338192
4,4,4,台中市綜合農產品批發市場,42682.312865
5,5,5,永靖鄉綜合農產品批發市場,56868.415441
6,6,6,溪湖鎮綜合農產品批發市場,84505.971098
7,7,7,南投市綜合農產品批發市場,2919.810496
8,8,8,西螺鎮綜合農產品批發市場,379836.636637
9,9,9,高雄市綜合農產品批發市場,79209.460641


- Formulation4
$$
\begin{split}
 \mbox{min} \quad & C^t\sum_{i \in I}\sum_{j \in J}D_{ij}(My_{ij} + x_{ij}) + \sum_{j \in J} d_j \quad \mbox{(carbon footprint)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & \sum_i x_{ij} \geq Q^d_j \quad \forall j \in J \quad \mbox{(lower limit of demand quantity)}\\
 & d_j \geq C_j - \sum_{i \in I}x_{ij} \quad \forall j \in J \\
 & d_j \geq \sum_{i \in I}x_{ij} - C_j \quad \forall j \in J \\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$


In [106]:
D

,新北市,台北市,桃園市,台中市,台南市,高雄市,宜蘭縣,新竹縣,苗栗縣,彰化縣,南投縣,雲林縣,嘉義縣,屏東縣,台東縣,花蓮縣,基隆市,新竹市,嘉義市,桃園縣
0,0.000,0.003,42.823,170.535,313.911,358.839,62.858,79.306,117.277,186.964,238.516,228.744,271.751,382.391,317.048,155.954,27.893,85.772,257.907,42.728
1,0.000,0.003,42.823,170.535,313.911,358.839,62.858,79.306,117.277,186.964,238.516,228.744,271.751,382.391,317.048,155.954,27.893,85.772,257.907,42.728
2,11.127,11.130,31.816,158.900,302.276,347.203,70.750,68.267,105.642,175.328,216.388,217.109,260.116,370.756,324.940,163.847,30.098,74.733,246.271,31.721
3,44.193,44.196,4.016,130.903,274.278,319.206,103.057,40.270,77.644,147.331,188.391,189.111,232.119,342.758,476.367,196.154,58.571,46.736,218.274,3.921
4,164.690,164.692,137.656,0.000,154.034,198.962,222.960,96.079,57.807,27.087,54.103,68.867,111.874,220.274,353.882,316.056,185.738,94.463,97.015,137.561
5,199.473,199.475,172.439,46.528,117.162,162.089,257.743,130.862,92.590,22.542,61.397,31.995,75.002,185.642,319.250,350.839,220.521,129.246,61.158,172.344
6,193.115,193.117,166.081,40.170,118.541,163.468,251.385,124.504,86.232,16.184,77.351,33.374,76.381,187.021,320.629,344.481,214.163,122.888,62.537,165.986
7,207.661,207.663,180.706,30.635,138.342,183.987,265.931,139.128,90.819,40.615,53.691,63.020,97.487,192.504,326.112,210.713,228.788,133.220,69.245,180.610
8,215.161,215.164,188.127,62.216,100.610,145.538,273.431,146.550,108.279,38.230,99.397,9.296,58.450,169.090,302.699,366.528,236.210,144.934,44.606,188.032
9,352.581,352.583,325.547,199.635,49.564,0.000,410.851,283.970,245.698,175.650,235.563,136.072,110.620,32.328,165.768,331.925,373.629,282.353,114.056,325.451
